Goal of this notebook:

Perform feature selection on our dataset.

Strategy:

Iterate over each project and execute the feature selection

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import warnings
import classifier_utils
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [4]:
projects = projects[:6]

In [5]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=100, max_features=0.3, min_samples_leaf=1)

# Tree-based feature selection

Uses the feature_importances_ attribute from the Random Forest model to select the most important features. It uses the mean of the importances of all features as a threshold.

In [6]:
import importlib
importlib.reload(classifier_utils)
results_tree, attributes_record_tree = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'tree')

In [7]:
results_tree

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759,129.000,33.0,0.740,0.734,-0.009
1,apache__directory-server,652,96.000,19.0,0.939,0.925,-0.015
2,android__platform_frameworks_base,2460,566.000,54.0,0.818,0.815,-0.004
3,freenet__fred,1012,134.000,47.0,0.679,0.680,0.003
4,alexo__wro4j,1368,107.000,30.0,0.586,0.577,-0.015
5,apache__lucene-solr,974,124.000,42.0,0.650,0.639,-0.017
6,Overall,7225,192.667,37.5,0.735,0.728,-0.010


# Recursive feature elimination:

First, the estimator is trained on the initial set of features and the importance of each feature is obtained. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.
Currently we are discarding 1 feature per step, using a 5-fold-cross-validation to calculate the accuracy on each step.

In [8]:
import importlib
importlib.reload(classifier_utils)
results_recursive, attributes_record_rec = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'recursive')

In [9]:
results_recursive

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759,129.000,120.000,0.740,0.748,0.030
1,apache__directory-server,652,96.000,41.000,0.939,0.937,-0.002
2,android__platform_frameworks_base,2460,566.000,138.000,0.818,0.816,-0.002
3,freenet__fred,1012,134.000,37.000,0.679,0.685,0.018
4,alexo__wro4j,1368,107.000,101.000,0.586,0.593,0.018
5,apache__lucene-solr,974,124.000,107.000,0.650,0.651,0.003
6,Overall,7225,192.667,90.667,0.735,0.738,0.012


# IGAR
Selects attributes based on the ranking of their information gain.
Information gain measures the ability of a feature to separate the target classes. The greater the information gain, the better its importance for classification tasks.

Information Gain = Entropy(overall) - Entropy(attribute)

The algorithm has an input value 'n' that is used to select the 'n' attributes with the greatest information gain among all attributes. In this notebook we use n = 65, which was the found in the notebook IGAR_tuning.ipynb.

In [10]:
import importlib
importlib.reload(classifier_utils)
results_IGAR, attributes_rec_IGAR = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'IGAR')

In [11]:
results_IGAR

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759,129.000,65.0,0.740,0.739,-0.002
1,apache__directory-server,652,96.000,65.0,0.939,0.936,-0.003
2,android__platform_frameworks_base,2460,566.000,65.0,0.818,0.814,-0.005
3,freenet__fred,1012,134.000,65.0,0.679,0.674,-0.007
4,alexo__wro4j,1368,107.000,65.0,0.586,0.580,-0.009
5,apache__lucene-solr,974,124.000,65.0,0.650,0.649,-0.002
6,Overall,7225,192.667,65.0,0.735,0.732,-0.005


## Comparison

In [12]:
df_inner = pd.merge(results_tree, results_recursive, on='project', how='inner', suffixes=('_tree', '_rec'))
df_inner_igar = results_IGAR.add_suffix("_IGAR").rename(columns={"project_IGAR": "project"})
df_inner = pd.merge(df_inner, df_inner_igar, on='project', how='inner')
df_inner.to_csv('feature_selection_comparison.csv', index=False)

accuracy_inner = df_inner.filter(regex=("project|accuracy.*")).copy()
accuracy_inner['improvement_tree'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_tree'], x['accuracy_tree']), axis=1)
accuracy_inner['improvement_rec'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_rec'], x['accuracy_rec']), axis=1)
accuracy_inner['improvement_IGAR'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_IGAR'], x['accuracy_IGAR']), axis=1)
accuracy_inner = accuracy_inner.round(3)
accuracy_inner

,project,accuracy_tree,accuracy_fs_tree,accuracy_rec,accuracy_fs_rec,accuracy_IGAR,accuracy_fs_IGAR,improvement_tree,improvement_rec,improvement_IGAR
0,Ramblurr__Anki-Android,0.740,0.734,0.740,0.748,0.740,0.739,-0.008,0.031,-0.001
1,apache__directory-server,0.939,0.925,0.939,0.937,0.939,0.936,-0.015,-0.002,-0.003
2,android__platform_frameworks_base,0.818,0.815,0.818,0.816,0.818,0.814,-0.004,-0.002,-0.005
3,freenet__fred,0.679,0.680,0.679,0.685,0.679,0.674,0.003,0.019,-0.007
4,alexo__wro4j,0.586,0.577,0.586,0.593,0.586,0.580,-0.015,0.017,-0.010
5,apache__lucene-solr,0.650,0.639,0.650,0.651,0.650,0.649,-0.017,0.003,-0.002
6,Overall,0.735,0.728,0.735,0.738,0.735,0.732,-0.010,0.011,-0.004


In [19]:
attributes_record = []
attributes_record.extend(attributes_record_tree)
attributes_record.extend(attributes_record_rec)
attributes_record.extend(attributes_rec_IGAR)
attributes_record_df = pd.DataFrame(attributes_record, columns=['project', 'attribute', 'information_gain', 'method'])
attributes_record_df.to_csv('attributes_record.csv', index=False)

## Ranking of features selected by tree method

Counts in how many projects the respective feature was selected using the tree method.

In [21]:
ranking_tree = classifier_utils.get_attribute_selection_ranking(attributes_record, 'tree')
ranking_tree.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
3,right_lines_removed,5.0,1.979777,3.0
2,right_lines_added,5.0,1.978852,3.0
29,Changed files 1,5.0,1.977518,3.0
30,Changed files 2,5.0,1.974373,3.0
5,keyword_fix,5.0,1.973496,3.0
27,Commits 1,5.0,1.962667,3.0
9,keyword_add,5.0,1.961819,3.0
1,left_lines_removed,5.0,1.959698,3.0
11,keyword_use,5.0,1.948103,3.0
42,Different devs,3.0,1.919683,2.0


## Ranking of features selected by recursive method

Counts in how many projects the respective feature was selected using the recursive method.

In [22]:
ranking_recursive = classifier_utils.get_attribute_selection_ranking(attributes_record, 'recursive')
ranking_recursive.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
205,alex.objelean@gmail.com,5.0,2.091704,1.0
3,right_lines_removed,25.0,1.979777,11.0
2,right_lines_added,25.0,1.978852,11.0
35,Commits 1,25.0,1.962667,11.0
12,keyword_add,25.0,1.961819,11.0
1,left_lines_removed,25.0,1.959698,11.0
32,Different devs,25.0,1.849775,11.0
21,chunkRelSize,30.0,1.833109,13.5
23,fileSize,30.0,1.831622,13.5
29,Merge isolation time,30.0,1.828842,13.5


## Ranking of features selected by IGAR method

Counts in how many projects the respective feature was selected using the IGAR method. 

The information gain column is an average among all projects.

In [23]:
ranking_IGAR = classifier_utils.get_attribute_selection_ranking(attributes_record, 'IGAR')
ranking_IGAR.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

,attribute,count_selected,average_information_gain,average_ranking
0,chunkRelSize,6.0,1.833109,3.500000
37,fileSize,6.0,1.831622,3.500000
3,Merge isolation time,6.0,1.828842,3.500000
1,Changed files 2,6.0,1.820250,3.500000
39,fileCC,6.0,1.819643,3.500000
22,Commits 1,6.0,1.817900,3.500000
19,Changed files 1,6.0,1.817638,3.500000
16,keyword_add,6.0,1.817193,3.500000
5,Branching time,6.0,1.816977,3.500000
2,left_lines_removed,6.0,1.815426,3.500000
